# Ladda ned PDF regeringsuppdrag

Sidan https://www.regeringen.se/regeringsuppdrag/ anropar en [JSON-fil](https://www.regeringen.se/Filter/GetFilteredItems?filterType=Taxonomy&filterByType=FilterablePageBase&preFilteredCategories=1342&rootPageReference=0&page=1&pageSize=500&displayLimited=false&sortAlphabetically=False) dynamiskt vid vaje anrop. Genom att manipulera `pageSize` i URL:en kan man plocka ut 500 länkar åt gången och sedan skrapa länkens HTML (JSON-filen innehåller HTML som värde) och därifrån hämta länken till PDF-filen som man sedan laddar ned.

Not: Med tanke på att JSON-filen verkar vara generell för allt möjligt innehåll kan man nog ladda ned fler PDF-filer än bara regeringsuppdrag (se exempelvis `preFilteredCategories=1342` i URL:en). Har dock inte testat.

In [1]:
from urllib import request
from lxml import html
from bs4 import BeautifulSoup
import ssl

# skrapar html
def scrapewebpage(url):
    UseSSL = True  # Om du får SSLError, ändra detta till False.
    if UseSSL:
        web = request.urlopen(url)
    else:
        web = request.urlopen(url, context=ssl._create_unverified_context())
    if web.getcode() == 200:
        return(web.read())
    else:
        print("Error %s reading %s" % str(web.getcode()), url)

# Utforska

Se vad JSON-filen innehåller för något smått och gott.

In [2]:
# skrapa json-fil
data = scrapewebpage("https://www.regeringen.se/Filter/GetFilteredItems?filterType=Taxonomy&filterByType=FilterablePageBase&preFilteredCategories=1342&rootPageReference=0&page=1&pageSize=500&displayLimited=false&sortAlphabetically=False")

In [3]:
# läs in json
import json
j = json.loads(data)

# nycklar i json-datan
for key in j.keys():
    print(key)

Message
TotalCount


In [4]:
# regeringsuppdrag totalt
j["TotalCount"]

2193

In [5]:
# plocka ut alla länkar på sidan från j["Message"] som innehåller html
soup = BeautifulSoup(j["Message"], "lxml-xml")
links = soup.find_all("a", "readmore")
len(links)

500

In [11]:
# kolla hur url:erna till regeringsuppdragen ser ut
url = links[0].get("href")
url

'/regeringsuppdrag/2018/06/uppdrag-att-samordna-det-nationella-klimatanpassningsarbetet-for-den-byggda-miljon/'

In [9]:
# hämta egeringsuppdragssidornas html
html = scrapewebpage("https://regeringen.se" + url)

In [22]:
# kolla var ".pdf" förekommer
pos = str(html).lower().find(".pdf")
html[pos-1000:pos+1000]

b''

# Försök parsa PDF-fil och metadata från HMTL

Nu när vi vet ungefär vad JSON-filen innehåller och hur HTML-sidorna är strukturerade, skapa funktioner för att hantera pdf-filernas namn, hämta ut metadata m.m.

In [54]:
# skapa några helper-funktioner
import urllib.request
from urllib.parse import urlparse
import os

# ta ut pdf-filnamnet (antar att första träffen på ".pdf" är rätt)
def get_pdf(pdf_soup):
    for a in pdf_soup.find_all("a"):
        if a != None:
            url = a.get("href")
            if url.lower().find(".pdf") > 0:
                return("https://regeringen.se" + url)
    # okej, hittade inget .pdf - gör då några desperata gissningar var pdf-filen finns
    url = [div.find("a") for div in pdf_soup.findAll("", "col-1")]
    if url != None:
        try:
            return("https://regeringen.se" + url[0].get("href"))
        except:
            pass
    # hittade verkligen inget, ge upp och gråt
    return("")

# ta ut pdf-filnamnet från url (släng på ".pdf" på slutet om det saknas)
def get_filename(url):
    if url.find("/") > 0:
        filename = url.rsplit("/", 1)[1]
        if not filename.lower().endswith(".pdf"):
            filename = filename + ".pdf"
        return(filename)

# ladda ned pdf
def download_pdf(url):
    urllib.request.urlretrieve(url, get_filename(url))

    
# ta ut lista över kategorier från metadatan
def get_metadata_categories(pdf_soup):
    l = list()
    ul = pdf_soup.find("", "block--politikomrLinks")
    for li in ul.find_all("li"):
        for a in li.find_all("a"):
            l.append(a.get_text())
    return(l)
    
# ta ut övrig metadata från regeringsuppdragets sida (titel, diarienummer, beskrivning, datum etc)
def get_metadata(url, pdf_soup):
    title = ""
    dnr = ""
    ingress = ""
    beskrivning = ""
    pub = ""
    h1 = pdf_soup.find("h1").get_text().strip()
    try:
        title = h1.split("\n")[0].strip()
        dnr = h1.split("\n")[1].strip()
        if len(dnr) > 0:
            dnr = dnr.replace("Diarienummer:", "").strip()
    except:
        title = h1
    try:
        ingress = pdf_soup.find("", "ingress has-wordExplanation").get_text().strip()
    except:
        pass
    try:
        explanation = pdf_soup.find("", "has-wordExplanation").get_text().strip()
    except:
        pass
    try:
        pub = pdf_soup.find("time").get("datetime")
    except:
        pass
    return({"titel": title, "dnr": dnr, "ingress": ingress, "beskrivning": explanation,  "pub": pub, 
            "pdf": get_filename(get_pdf(pdf_soup)), "url": url, "kategorier": get_metadata_categories(pdf_soup)})

In [55]:
# kolla att det verkar funka på ett regeringsuppdrag
url = "https://regeringen.se/regeringsuppdrag/2018/06/uppdrag-om-saker-och-effektiv-tillgang-till-grunddata/"
html = scrapewebpage(url)
pdf_soup = BeautifulSoup(html, "lxml-xml")
get_metadata(url, pdf_soup)

{'titel': 'Uppdrag om säker och effektiv tillgång till grunddata',
 'dnr': '',
 'ingress': '',
 'beskrivning': 'Regeringen gerBolagsverket, Lantmäteriet och Skatteverket i uppdrag att tillsammans lämna förslag som syftar till att skapa en säker och effektiv till\xadgång till grunddata, genom att bl.a. tydliggöra ansvaret för och öka standardi\xadseringen av sådan data.\nUppdraget ska slutredovisas till regeringen senast den 30 april 2019.',
 'pub': '07 juni 2018',
 'pdf': 'uppdrag-om-saker-och-effektiv-tillgang-till-grunddata.pdf',
 'url': 'https://regeringen.se/regeringsuppdrag/2018/06/uppdrag-om-saker-och-effektiv-tillgang-till-grunddata/',
 'kategorier': ['Digitaliseringspolitik',
  'Finansdepartementet',
  'Regeringen',
  'Regeringsuppdrag',
  'Statlig förvaltning']}

In [56]:
# vissa regeringsuppdrag har inget .pdf i slutet av filnamnet, som den här,
# kolla att den kan parsa ut pdf-filen ändå
url = "https://regeringen.se/regeringsuppdrag/2018/06/uppdrag-till-arbetsformedlingen-om-en-modell-for-upphandlad-matchning-till-etableringsjobb/"
html = scrapewebpage(url)
pdf_soup = BeautifulSoup(html, "lxml-xml")
get_metadata(url, pdf_soup)

{'titel': 'Uppdrag till Arbetsförmedlingen om en modell för upphandlad matchning till etableringsjobb',
 'dnr': 'A2018/01212/A',
 'ingress': 'Regeringen uppdrar åt Arbetsförmedlingen att analysera förutsättningarna för och utreda den närmare utformningen av en modell för upphandlad matchning till etableringsjobb. Uppdraget ska genomföras i samråd med LO, Unionen och Svenskt Näringsliv.',
 'beskrivning': 'Arbetsförmedlingen ska analysera hur tjänsten Stöd och matchning kan utvecklas och hur tjänsten kan användas tillsammans med andra insatser för att skapa en effektiv matchning till etableringsjobb. Arbetsförmedlingen ska utifrån detta perspektiv analysera dimensioneringen av tjänsten.\nUtifrån analysen ska Arbetsförmedlingen redovisa hur utformningen av en upphandlad modell för matchning till etableringsjobb skulle kunna se ut. Om det bedöms finnas behov av författningsändringar för att kunna utforma en effektiv modell för upphandlad matchning till etableringsjobb ska sådant förslag re

In [53]:
# denna har inte ens en pdf kopplad till sig
url = "https://regeringen.se/regeringsuppdrag/2018/05/uppdrag-att-inom-ramen-for-livsmedelsstrategin-vidta-atgarder-for-framjande-av-produktion-konsumtion-och-export-av-ekologiska-livsmedel/"
html = scrapewebpage(url)
pdf_soup = BeautifulSoup(html, "lxml-xml")
get_metadata(url, pdf_soup)

{'titel': 'Uppdrag att, inom ramen för livsmedelsstrategin, vidta åtgärder för främjande av produktion, konsumtion och export av ekologiska livsmedel',
 'dnr': 'Diarienummer: N2018/02711/JM',
 'ingress': 'Regeringen uppdrar åt Statens jordbruksverk att efter samråd med berörda myndigheter, företrädare för företag och organisationer inom hela livsmedelskedjan samt konsument- och miljöorganisationer, vidta lämpliga åtgärder som ska syfta till att nå regeringens uppsatta inriktningsmål för ekologisk produktion och konsumtion, som innebär att 30 procent av den svenska jordbruksmarken ska utgöras av certifierad ekologisk jordbruksmark år 2030 och att 60 procent av den offentliga livsmedelskonsumtionen ska utgöras av certifierade ekologiska produkter år2030.',
 'beskrivning': 'En årlig ekonomisk redovisning av uppdraget ska lämnassenast den 28 februari till Regeringskansliet (Näringsdepartementet).',
 'pub': '29 maj 2018',
 'pdf': None,
 'url': 'https://regeringen.se/regeringsuppdrag/2018/05

# Skapa sqlite3 databas

Någonstans måste vi ju spara allt.

In [60]:
import sqlite3 as lite
con = lite.connect("regeringsuppdrag.db")
cur = con.cursor()    

# skapa databas om den inte redan finns
cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
counter = cur.fetchall()
if len(counter) < 2:
    cur.execute("CREATE TABLE regeringsuppdrag (id INTEGER PRIMARY KEY AUTOINCREMENT, titel TEXT, dnr TEXT, ingress TEXT, beskrivning TEXT, pub TEXT, pdf TEXT, url TEXT, kategorier TEXT)")

# gör om json-kategorier till komma-separerad lista
def convert_to_string(l):
    return(", ".join(str(x) for x in l))

# dumpa json till sqlite
def write_metadata(j):
    with con:
        cur = con.cursor()
        cur.execute("INSERT INTO regeringsuppdrag (titel, dnr, ingress, beskrivning, pub, pdf, url, kategorier) VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
                    (j["titel"], j["dnr"], j["ingress"], j["beskrivning"], j["pub"], j["pdf"], j["url"], convert_to_string(j["kategorier"])))

# Skrapa allt

Nu verkar allt fungera att hämta.

Detta är huvudfunktion för att skrapa alla pdf:er. Notera att det är någon sekunds fördröjning:

In [61]:
import csv
import datetime
import time
import sys
import math
i = 0
errors = 0

print("Start " + str(datetime.datetime.now().time()))
        
# 2190 regeringsuppdrag / 500 per sida = 5 sidor,
# vi behöver med andra ord köra for-loopen 5 gånger
for page in range(1, math.ceil(int(j["TotalCount"]) / 500) + 1): 
    print("Sida {0}".format(page))
    # skrapa 500 resultat åt gången och ta ut alla (förhoppningsvis) 500 länkar
    data = scrapewebpage("https://www.regeringen.se/Filter/GetFilteredItems?filterType=Taxonomy&filterByType=FilterablePageBase&preFilteredCategories=1342&rootPageReference=0&page=" + str(page) + "&pageSize=500&displayLimited=false&sortAlphabetically=False")
    j = json.loads(data)    
    soup = BeautifulSoup(j["Message"], "lxml-xml")
    links = soup.find_all("a", "readmore")
    
    # skrapa länk för länk med regeringsuppdrag
    print("Laddar ned pdf", end=" ")
    for a in links:
        if a != None:
            url = a.get("href")
            try:
                # ta ut varje sida med regeringsuppdrag & ladda ned pdf
                i += 1
                html = scrapewebpage("https://regeringen.se" + url)
                pdf_soup = BeautifulSoup(html, "lxml-xml")
                pdf_url = get_pdf(pdf_soup)
                write_metadata(get_metadata(url, pdf_soup))
                if pdf_url != "":
                    #print("Laddar ned https://regeringen.se" + pdf_url)
                    print(i, end=" ")
                    download_pdf(pdf_url)
                    time.sleep(2.0) # Fördröjning för att vara snäll mot regeringen (maskinerna kommer ta över världen ändå en vacker dag)
                else:
                    print()
                    print("hittade ingen pdf: https://regeringen.se" + url)
            except KeyboardInterrupt:
                con.close()
                sys.exit()
            except:
                errors += 1
                print()
                print("Fel: https://regeringen.se" + url + " " + str(sys.exc_info()[0]))
                  
con.close()
print()
print()
print("Klart " + str(datetime.datetime.now().time()))
print("{0} pdf:er nedladdat, {1} fel".format(i, errors))

Start 19:10:42.065370
Sida 1
Laddar ned pdf 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 
hittade ingen pdf: https://regeringen.se/regeringsuppdrag/2018/05/uppdrag-att-inom-ramen-for-livsmedelsstrategin-vidta-atgarder-for-framjande-av-produktion-konsumtion-och-export-av-ekologiska-livsmedel/
17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216

# Städa upp databasen

Av någon anledning sparas inte hela URL:en i databasen, och publiceringsdatumet är i olika format. Fixa dessa saker här.

In [3]:
# först, skapa kopia av databasen ifall något går snett
!copy regeringsuppdrag.db regeringsuppdrag_original.db

        1 file(s) copied.


In [98]:
import sqlite3 as lite
con = lite.connect("regeringsuppdrag.db")
cur = con.cursor()

# lägg på domän på url som av någon anledning inte kom med tidigare
cur.execute("UPDATE regeringsuppdrag SET url = 'https://regeringen.se' || url;")

# skapa nytt fält för datum
cur.execute("ALTER TABLE regeringsuppdrag ADD pub_date TEXT;")

# skapa nytt fält för år
cur.execute("ALTER TABLE regeringsuppdrag ADD pub_year TEXT;")

con.commit()
con.close()

In [28]:
# behövs för att parsa datum på olika språk (däribland svenska)
!pip install dateparser

  Running setup.py bdist_wheel for tzlocal: started
  Running setup.py bdist_wheel for tzlocal: finished with status 'done'
  Stored in directory: C:\Users\Peter\AppData\Local\pip\Cache\wheels\15\ae\df\a67bf1ed84e9bf230187d36d8dcfd30072bea0236cb059ed91
Successfully built tzlocal


mysql-connector-python 8.0.11 requires protobuf>=3.0.0, which is not installed.
distributed 1.21.8 requires msgpack, which is not installed.


In [100]:
# kolla först att det fungerar att använda dateparser
import datetime
import dateparser

# parsa datum
con = lite.connect("regeringsuppdrag.db")
cur = con.cursor()
cur.execute("SELECT pub FROM regeringsuppdrag LIMIT 10;")
pub = cur.fetchall()
con.close()
for row in pub:
    dt = dateparser.parse(row[0], languages=["sv"])
    print(row[0] + " --> " + str(dt.date()))

15 juni 2018 --> 2018-06-15
15 juni 2018 --> 2018-06-15
13 juni 2018 --> 2018-06-13
11 juni 2018 --> 2018-06-11
11 juni 2018 --> 2018-06-11
07 juni 2018 --> 2018-06-07
07 juni 2018 --> 2018-06-07
07 juni 2018 --> 2018-06-07
05 juni 2018 --> 2018-06-05
05 juni 2018 --> 2018-06-05


In [107]:
# dateparser fungerade bra, då kan vi skapa ett nytt datumfält för hela tabellen
con = lite.connect("regeringsuppdrag.db")
cur = con.cursor()
cur.execute("SELECT id, pub FROM regeringsuppdrag;")
pub = cur.fetchall()
for row in pub:
    dt = dateparser.parse(row[1], languages=["sv"])
    cur2 = con.cursor()
    cur2.execute("UPDATE regeringsuppdrag SET pub_date=?, pub_year=? WHERE id=?", (dt.date(), dt.year, row[0]))

con.commit()
con.close()